# 4. Regresión Lineal

### Generar conjuntos de datos

In [ ]:
# Carga bibliotecas útiles
import numpy as np
from sklearn import datasets
import matplotlib.pyplot as plt
%matplotlib inline

### Genera conjunto de datos

In [ ]:
N = 1500

In [ ]:
# Blobs aniisotrópicos
# Covarianza con dirección preferencial
X, Lblobsa = datasets.make_blobs(n_samples=N, random_state=170)
Xblobsa = np.dot(X, [[0.6, -0.6], [-0.4, 0.8]])

# Blobs de distinta varianza
Xblobsv, Lblobsv = datasets.make_blobs(n_samples=N, cluster_std=[1.0, 2.5, 0.5],
                             random_state=170)
plt.figure(figsize=(15,5))
for X, l, i in zip((Xblobsa,Xblobsv),(Lblobsa,Lblobsv),range(1,4)):
  plt.subplot(1,3,i)
  plt.grid(True)
  plt.scatter(X[l==0, 0], X[l==0, 1], s=20, color='red');
  plt.scatter(X[l==1, 0], X[l==1, 1], s=20, color='blue');
  plt.scatter(X[l==2, 0], X[l==2, 1], s=20, color='green');

La regresión lineal es un método que permite estimar los hiperparámetros del hiperplano que mejor describa un conjunto de datos.

Sea $X=\{x_1, x_2,..., x_m\}: x_i\in\mathbb{R}^n, \forall i,m,n\in\mathbb{N}$ el conjunto de vectores independientes obtenidos por medio de una medición, mientras que $Y=\{y_1, y_2,..., y_m\}:\in\mathbb{R}$ corresponden a las variables medidas dependientes de $X$.

Dada la ecuación general de un hiperplano
$$y(x)=w^T x+b: w,x\in\mathbb{R}^n, y,b\in\mathbb{R},$$

donde $w$ y $b$ son estimados a partir de $X$ y $Y$ por medio de un proceso de optimización y corresponden al vector normal al hiperplano estimado y al sesgo, respectivamente.


In [ ]:
"""
 @note: Únicamente se toman en cuenta uno de los blobs isotrópicos
 en este casos se muestra un conjunto de datos y como se vería una
 mala descripción de sus datos aproximados con y(x) = x
"""
x, y = Xblobsa[Lblobsa==0, 0], Xblobsa[Lblobsa==0, 1]

plt.figure(figsize=(10,5))
plt.plot(x,y,'o',markersize=5,color='orange')
plt.plot(x,x,'-',color='black')

plt.grid(True)

In [ ]:
"""
 @note: Al medir las distancias de x_i con respecto al hiperplano y acumularlas
 se obtiene el error cuadrático
"""
x, y = Xblobsa[Lblobsa==0, 0], Xblobsa[Lblobsa==0, 1]

plt.figure(figsize=(10,5))
plt.plot(x,y,'o',markersize=5,color='orange')
plt.plot(x,x,'-',color='black')

for i,j in zip(x,y):
    plt.arrow( i,j,0,i-j )

plt.grid(True)

In [ ]:
y_est = x
ec    = np.dot( (y-y_est).T, (y-y_est) )

print (ec)

La regresión lineal basada en LDA se fundamenta en minimizar el error cuadrático medio acumulado de las distancias $x_i$ al hiperplano. La fundamentación teórica queda fuera de los alcances de este curso, es por ello que se utilizarán las bibliotecas de scikit-learn que realizan dicho proceso de regresión.

In [ ]:
#Importar bibliotecas para regresión

from sklearn              import linear_model as LRM
from sklearn.linear_model import LassoLarsCV
from sklearn.svm          import SVR
from sklearn.metrics      import mean_squared_error, r2_score

In [ ]:
# Crear objeto de regresión lineal y entrenarlo

model = LRM.LinearRegression()
#x[:,np.newaxis] se requiere porque se necesita un vector columna para el entrenamiento
#si x.shape es de (m,n) para n>1 -> x[:,np.newaxis] no es necesario
model.fit( x[:,np.newaxis],y) 

#en este caso se entrena y se predice con los mismos datos
#únicamente con fines ilustrativos
y_est = model.predict( x[:,np.newaxis] )

In [ ]:
ec = np.dot( (y-y_est).T, (y-y_est) )

#se observa que el error cuadrático se reduce drásticamente
print (ec)

In [ ]:
plt.figure(figsize=(10,5))
plt.plot(x,y,'o',markersize=5,color='orange')
plt.plot(x,y_est,'-',color='black')

for i,j,k in zip(x,y,y_est):
    plt.arrow( i,j,0,k-j )

plt.grid(True)

In [ ]:
"""
 @note: Se utilizan métricas más robustas
 @mse: Error cuadrático medio
 @r2: Coeficiente de correlación de Pearson, miestras más cercano a -1 o 1 significa 
 una anticorrelación o correlación perfecta, mientras más cercano a 0 no existe correlación
 de ningún tipo. Dependiendo de la aplicación un buen coeficiente de correlación puede
 ser r2>|0.5|
"""
print ( "ECM: %4.4f"%(mean_squared_error             (y,y_est) )  )
print ( "Coeficiente de correlación: %4.4f"%(r2_score(y,y_est) )  )

### Ejemplo con datos reales
La base de datos de diabetes constá de 441 registros de 10 rasgos *data*$\in\mathbb{R}^{442\times10}$ y un rasgo de progresión (*target*).

In [ ]:
data     = datasets.load_diabetes()
diabetes = data['data']
targets  = data['target']

In [ ]:
"""
 @note: Los 442 datos se parten en un conjunto de entrenamiento y otro de prueba
 el primero de 342 muetras y el otro 100.
 @note: para evitar sesgo se seleccionan aleatoriamente los datos de cada conjunto
"""
N   = 100
idx = np.arange( diabetes.shape[0] )
np.random.shuffle(idx)

XTrain = diabetes[ idx[:-N] ]
YTrain = targets [ idx[:-N] ]

XTest = diabetes[ idx[-N:] ]
YTest = targets [ idx[-N:] ]

In [ ]:
"""
 @note: Se entrena la regresión lineal y se evalúa su desempeño
"""
model = LRM.LinearRegression()
model.fit( XTrain,YTrain )

L = model.predict( XTest )

print ( "ECM: %4.4f"%(mean_squared_error             (YTest,L) )  )
print ( "Coeficiente de correlación: %4.4f"%(r2_score(YTest,L) )  )

Se observa que el $R^2$ apenas supera el 0.5, lo cual en aplicaciones médicas no es funcional, aunque indica una leve correlación, pero sin grandes implicaciones diagnósticas.

En este caso no es posible graficar la regresión ya que está en $\mathbb{R}^{11}$, sin embargo podría realizarse con scatter_plots 

## Regresión *Least absolute shrinkage and selection operator* (LASSO)

Es una regresión lineal tipo LDA, pero que pondera los pesos $w$ por la norma $L_1$ lo que se traduce en hacer la $j$-ésima componente del vector $w$, $$w[j]:j\subset\{1,2,..., n\},$$ exactamente igual a cero

In [ ]:
"""
 @note: En este caso se entrena LASSO con el algortimo LARS que converge más rápidamente
 además se utiliza validación cruzada la optimización del parámetro de *shrinkage*
"""
model = LassoLarsCV(max_iter=100000,n_jobs=4,cv=8)
model.fit( XTrain,YTrain )

L = model.predict( XTest )

print ( "ECM: %4.4f"%(mean_squared_error             (YTest,L) )  )
print ( "Coeficiente de correlación: %4.4f"%(r2_score(YTest,L) )  )

In [ ]:
# Aquí se muestra como se relacionan la Y de prueba y la Y estimada por
# el método de regresión lineal
plt.figure(figsize=(10,5))
plt.plot(YTest,L,'o',color='orange',markersize=8)
plt.grid(True)

El método LASSO selecciona nativamente los rasgos que más contribuyen a la regresión haciendo cero aquellas que no lo hacen.

In [ ]:
"""
 @note: Se crean tres variables eta0, eta1, eta2 que se agregan a XTrain para ser
 fuentes de ruido.
 @note: Se entrenan dos modelos, uno con regresión lineal convencional y otro con LASSO.
"""

eta0 = np.ones(  XTrain.shape[0] )[:,np.newaxis]
eta1 = np.array( [0 , 1, 2  ]*114 )[:,np.newaxis]
eta2 = np.array( [-1, 1, 0  ]*114 )[:,np.newaxis]

np.random.shuffle( eta1 )
np.random.shuffle( eta2 )

XTrain_mod = np.concatenate( (eta0,XTrain,eta1,eta2),axis=1 )

model = LRM.LinearRegression()
model.fit( XTrain_mod,YTrain )

model2 = LassoLarsCV(max_iter=1000000,n_jobs=4,cv=100)
model2.fit( XTrain_mod,YTrain )

In [ ]:
"""
 @note: Como los resultados dependen en cada caso depende del conjunto de entrenamiento
 y la validación cruzada aquí se muestran los obtenidos en este caso. Se observa que
 LDA y LASSO detectan perfetamente el vector de unos (componente 0). LASSO además de detectar
 los vectores de ruido en las componentes 10 y 11, también hace cero tres componentes más, es
 decir, que no contribuyen a la regresión.
 @output: >>> 
LDA,   LASSO
0.0000,	0.0000
-76.9286,	0.0000
-228.6376,	-121.6773
486.1239,	481.9079
327.5262,	245.8395
-183.6908,	0.0000
55.2152,	0.0000
-123.8803,	-177.8817
114.9689,	0.0000
566.7349,	484.7121
26.2757,	0.0000
2.4359,	0.0000
-3.4907,	0.0000
"""

print ('LDA,   LASSO')
for i,j in zip(model.coef_,model2.coef_):
    print ('%4.4f,\t%4.4f'%(i,j) )

In [ ]:
"""
 @note: Selección de rasgos (componentes) relevantes detectadas por LASSO
 @note: Salida para este caso
 @output: >>> array([2, 3, 4, 7, 9])
"""
idx = np.arange( model2.coef_.shape[0] )
idx = (model2.coef_!=0).astype('int') * idx
idx = idx[idx!=0]

In [ ]:
"""
 @note: De esta forma XTrain_mod queda únicamente de cinco rasgos, para este caso
 @output: >>> (342, 5)
"""
XTrain_mod[:,idx].shape

## Tarea Rapsberry Pi.
 - Descargar la base de datos de [eficiencia energética](http://archive.ics.uci.edu/ml/datasets/Energy+efficiency)    
 - Estimar la regresión lineal para *Heating Load* y para *Cooling Load* por separado
 - ¿Todos los rasgos son relevantes?
 
**Nota:** No olvide separar sus datos en conjuntos de entrenamiento y prueba